# How the algorithm works
### label real data using a service like roboflow/label real data usingg an old model
### put the labeled data (masks, images) in the folder temp
### run this line to get the polygons from the masks 


In [24]:
%run masks_to_polygons.py

### By this stage you should have three folders inside temp (images, labels, masks)
### run the following function (split_img_label), this function will take the temp data and transform it into a data set of images and lables each having their train and validation data set

In [3]:
from ultralytics import YOLO
from tqdm import tqdm
import subprocess

In [35]:
def split_img_label(data_train,data_test,folder_train,folder_test):
    
    # os.mkdir(folder_train)
    # os.mkdir(folder_test)
    main_dir = "dataset"
    sub_dirs = ["images", "labels"]
    train_val_dirs = ["train", "val"]
    
    # Create the main dataset directory
    os.makedirs(main_dir, exist_ok=True)
    
    # Create subdirectories and train/val folders
    for sub_dir in sub_dirs:
        # Create the folder for images and labels
        sub_dir_path = os.path.join(main_dir, sub_dir)
        os.makedirs(sub_dir_path, exist_ok=True)
        
        for train_val in train_val_dirs:
            # Create the train and val folders inside images and labels
            os.makedirs(os.path.join(sub_dir_path, train_val), exist_ok=True)

    
    train_ind=list(data_train.index)
    test_ind=list(data_test.index)
    folder_train_images="dataset\\images\\train"
    folder_train_labels="dataset\\labels\\train"
    folder_test_images="dataset\\images\\val"
    folder_test_labels="dataset\\labels\\val"
  
    # Train folder
    for i in tqdm(range(len(train_ind))):
        source ='.\\'+ "tmp\\images" + '\\'  +data_train[train_ind[i]].split('/')[3]
        destination = '.\\'+ folder_train_images + '\\'  +data_train[train_ind[i]].split('/')[3]
        command = f'copy "{source}" "{destination}"'
        os.system(command)
        source ='.\\'+ "tmp\\labels" + '\\'  +data_train[train_ind[i]].split('/')[3].split('.png')[0]+'.txt'
        destination = '.\\'+ folder_train_labels + '\\'  +data_train[train_ind[i]].split('/')[3].split('.png')[0]+'.txt'
        command = f'copy "{source}" "{destination}"'
        os.system(command)
     
# Execute the command
#     Test folder
    for j in tqdm(range(len(test_ind))):
        source ='.\\'+ "tmp\\images" + '\\'  +data_test[test_ind[j]].split('/')[3]
        destination = '.\\'+ folder_test_images + '\\'  +data_test[test_ind[j]].split('/')[3]
        command = f'copy "{source}" "{destination}"'
        os.system(command)
        source ='.\\'+ "tmp\\labels" + '\\'  +data_test[test_ind[j]].split('/')[3].split('.png')[0]+'.txt'
        destination = '.\\'+ folder_test_labels + '\\'  +data_test[test_ind[j]].split('/')[3].split('.png')[0]+'.txt'
        command = f'copy "{source}" "{destination}"'
        os.system(command)


### the  following code is only responsible for creating a list of the names of the image and label files into an array and running the split function on that indexing. !Be careful of the image type as it requires a minor change to jpg if such files exists 

In [36]:
import pandas as pd 
import os 
import sklearn.model_selection
from sklearn.model_selection import train_test_split
import glob

PATH_Images = './tmp/images/'
PATH_Labels= './tmp/labels/'

list_img=[img for img in os.listdir(PATH_Images) if img.endswith('.png')==True]# you can change this extension to jpg
list_txt=[img for img in os.listdir(PATH_Labels) if img.endswith('.txt')==True]

path_img=[]

for i in range (len(list_img)):
    path_img.append(PATH_Images+list_img[i])
    
df=pd.DataFrame(path_img)

# split 
data_train, data_test, labels_train, labels_test = train_test_split(df[0], df.index, test_size=0.20, random_state=42)
folder_train_name="train"
folder_test_name="test"
# Function split 
split_img_label(data_train,data_test,folder_train_name,folder_test_name)


100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 36.75it/s]


### by this stage you should hava a ready to train data set

In [3]:
model = YOLO('yolov8n-seg.pt')  # в случае добучения - указать путь до предобученной модели. базовая скачается автоматически
model.train(data='yolo.yaml', epochs=50, device=[0], imgsz=640)


New https://pypi.org/project/ultralytics/8.3.18 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.3  Python-3.9.20 torch-2.0.1 CUDA:0 (NVIDIA GeForce GTX 1050, 2048MiB)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=yolo.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, 

KeyboardInterrupt: 

In [1]:
torch.cuda.is_available(): True
torch.cuda.device_count(): 1
os.environ['CUDA_VISIBLE_DEVICES']: 1,2,3

SyntaxError: illegal target for annotation (442746295.py, line 1)

### by this stage you should have a model somewhere in the folder (runs/segment/train%d)
### move the images that you want to annotate to the folder "predicted_masks/images"
### Folder Structure Explanation

To run this notebook successfully, ensure that there is a folder named `predicted_masks` in the same directory as the notebook. Inside the `predicted_masks` folder, there should be three subfolders:

1. **images**: This folder contains the images for predictions.
2. **masks**: This folder contains the mask predictions corresponding to the images.
3. **overlay**: This folder contains overlay images (a combination of the original images and their predicted masks).

The structure should look li

    ./predicted_masks 
                    ├── images (put your images here)
                    ├── masks
                    └── overlay
                 this:



### the following code is only responsible for overlaying the mask on top of the image, it has two versions one for png images and the other for jpg, but be aware that you have to call the right version depending on what you need in the function "*save_prediction_masks"

In [15]:
import torch
torch.cuda.empty_cache()

In [23]:
from PIL import Image
import numpy as np
import cv2

def overlay_mask(image_path, mask, alpha=0.8):
    # Load the image
    # image = Image.open(image_path).convert("RGB")
    image = Image.open(image_path).convert("RGBA")
    # resized_image = image.resize(384,480)

    # Load the mask and convert it to an array
    mask = Image.open(mask_path).convert("L")  # Convert to grayscale
    mask = np.array(mask)

    # Create a colored version of the mask
    colored_mask = np.zeros((*mask.shape, 4), dtype=np.uint8)  # RGBA
    colored_mask[:, :, 0] = 255  # Red channel (color of the mask)
    colored_mask[:, :, 3] = mask * 255  # Alpha channel based on the mask

    # Convert the colored mask to an image
    colored_mask_image = Image.fromarray(colored_mask, 'RGBA')

    # Overlay the mask on the image with specified transparency
    overlay = Image.alpha_composite(image, colored_mask_image)

    # Save the result to the specified path
    overlay_path = "./predicted_masks/overlay/" + image_path.split('/')[3]
    overlay.save(overlay_path)
    
    # Convert overlay to format compatible with OpenCV (BGR) for further processing (if necessary)
    overlay_cv = cv2.cvtColor(np.array(overlay), cv2.COLOR_RGBA2BGR)
    cv2.imwrite(overlay_path, overlay_cv)

# Example usage:
# overlay_mask('./predicted_masks/images/00435-4194690285.png', './predicted_masks/masks/00435-4194690285.png')

In [33]:
from PIL import Image
import cv2
import numpy as np

def overlay_mask_jpg(image_path, mask, alpha=0.5):
    overlay_path = "./predicted_masks/overlay/" + image_path.split('/')[3]
    
    img_pil = Image.open(image_path).convert("RGBA")
    overlay = Image.new('RGBA', img_pil.size, (255, 255, 255, 0))
    overlay_draw = ImageDraw.Draw(overlay)
    polygon = mask.xy[0]
    if len(polygon) >= 3:
        overlay_draw.polygon(polygon, outline=(0, 255, 0), fill=(0, 255, 0, 127))

        polygon_shapely = Polygon(polygon)
        centroid = polygon_shapely.centroid
        circle_radius = 5
        left_up_point = (centroid.x - circle_radius, centroid.y - circle_radius)
        right_down_point = (centroid.x + circle_radius, centroid.y + circle_radius)
        overlay_draw.ellipse([left_up_point, right_down_point], fill=(255, 0, 0))
    
    img_pil = Image.alpha_composite(img_pil, overlay)

    frame = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    cv2.imwrite(overlay_path, frame)

In [37]:
import os
from pathlib import Path
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
from pixel_size.utils import get_pixel_real_size
from weld_processing.read_mask import plot_mask_and_point,return_points_and_size
from PIL import Image, ImageDraw
from shapely.geometry import Polygon

def save_prediction_masks():
    PATH_Images="./predicted_masks/images"
    PATH_Masks="./predicted_masks/masks"
    
    list_img=[img for img in os.listdir(PATH_Images) if img.endswith('.png')==True]
    model_path = Path('runs/segment/train10/weights/best.pt')  # Adjust path as needed
    model = YOLO(model_path)
    for i in tqdm(list_img):
        img_path=PATH_Images+"/"+ i
        # Load the model (replace 'best.pt' with your actual model file name)
        
        # Load an image
        img = cv2.imread(img_path)

            # Resize the image
        
        H, W,_ = img.shape
        results = model(img,imgsz=480)
        
        mask= results[0].masks.data[0].cpu().numpy() * 255
        print(mask.shape[0]/mask.shape[1])
        # mask = cv2.resize(mask, (W, H))
        print(H/W)
    
        mask_path=PATH_Masks+'/'+ i
 
        cv2.imwrite(mask_path, mask)
        print(mask_path, img_path)
        overlay_mask_jpg(img_path,results[0].masks[0])

      

# # res = get_pixel_real_size(img_path)
# # print(res)
# # top_line_len,top_coords, bot_line_len,bot_coords = return_points_and_size('./output.png')
# # print(f'top_line_len={top_line_len,top_line_len*res[0]}mm, bot_line_len={bot_line_len,bot_line_len*res[0]}')

# # plot_mask_and_point('./output.png')
save_prediction_masks()


0it [00:00, ?it/s]


#### by this stage you should have your overlayed images and masks in the folder "predicted_masks"
#### you can then choose to add those newly annotated data to your data set and train a new iteration

In [7]:
import shutil
def add_to_dataset():
    path_images = './tmp/images/'
    path_masks= './tmp/masks/'
    path_masks_predicted='./predicted_masks/masks/'
    path_images_predicted='./predicted_masks/images/'
    path_overlay='./predicted_masks/overlay/'
    list_img=[img for img in os.listdir(path_overlay) if img.endswith('.jpg')==True]
    for i in tqdm(list_img):
        img_path_predicted=path_images_predicted+"/"+ i
        mask_path_predicted=path_masks_predicted+"/"+ i
          # Move predicted image to the image directory
        if os.path.exists(img_path_predicted):
            shutil.move(img_path_predicted, path_images)
        
        # Move predicted mask to the masks directory
        if os.path.exists(mask_path_predicted):
            shutil.move(mask_path_predicted, path_masks)
        
add_to_dataset()  

NameError: name 'os' is not defined

In [17]:

# model_path = Path('runs/segment/train3/weights/best.pt')  # Adjust path as needed
model_path = Path('runs/segment/train10/weights/best.pt')  # Adjust path as needed
model = YOLO(model_path)
img_path="./predicted_masks/images/10a.jpg"
img = cv2.imread(img_path)
results=model(img)
annotated_frame = results[0].plot()
annotated_frame
cv2.imshow("Yolov8Inference",annotated_frame)



0: 384x480 1 weld, 11.5ms
Speed: 3.0ms preprocess, 11.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 480)


In [3]:
import os

# Specify the folder path
folder_path = "./"

# List all files and folders
items = os.listdir(folder_path)

# Print all files and folders
for item in items:
    print(item)

.git
.gitignore
.ipynb_checkpoints
AI_GENERATED_DATA
annotating welding .ipynb
Annotations
annotation_helper.py
Assets
best.pt
coco.py
datasets
Demo - COTS Dataset - Multiple Objects.ipynb
Demo - COTS Dataset - Single Object.ipynb
Environment
Evaluation
LICENSE
masks_to_polygons.py
package-list.txt
pixel_size
predicted_masks
presentation.pdf
README.md
Report Aveen.ipynb
runs
tmp
toTrainOn
vgg.py
weld_processing
yolo.py
yolo.yaml
yolo11n.pt
YoloTraining.ipynb
yolov8n-seg.pt
yolov8n.pt
__pycache__
